# 🧠 Brain Tumor MRI Image Classification
Developed using **TensorFlow/Keras** and **PyTorch**

This notebook builds and evaluates deep learning models (Custom CNN & Transfer Learning) to classify brain tumors using MRI images. It includes data preprocessing, augmentation, training, evaluation, model comparison, and notes for deployment using Streamlit.

In [13]:
# 📦 Install & Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, MobileNetV2

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


In [2]:
import gdown

# Replace this with your actual sharable Google Drive file link
url = 'https://drive.google.com/drive/folders/1KQMtJytnS30zJUqNKbrK0FG6VVdWTFCD?usp=sharing'
gdown.download(url, 'Brain_Tumor_Dataset.zip', quiet=False)


/Users/shreyadas/tf-env/lib/python3.10/site-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/folders/1KQMtJytnS30zJUqNKbrK0FG6VVdWTFCD?usp=sharing
To: /Users/shreyadas/Downloads/Brain_Tumor_Dataset.zip
1.25MB [00:01, 1.03MB/s]


'Brain_Tumor_Dataset.zip'

In [17]:
print("Is zip file?", zipfile.is_zipfile(zip_path))


Is zip file? False


In [2]:
import gdown

# Convert Google Drive share link to direct download link:
# FILE ID = 1MQHKl2Q1fKV7Mle3uB2J6KXdZOmy0gB6
file_id = "1XrJR8wQoh5KtOOSnBhmEmpl4JlqQ3j50"
direct_url = f"https://drive.google.com/uc?id={file_id}"

# Download using gdown
gdown.download(direct_url, 'Brain_Tumor_Dataset.zip', quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1XrJR8wQoh5KtOOSnBhmEmpl4JlqQ3j50
From (redirected): https://drive.google.com/uc?id=1XrJR8wQoh5KtOOSnBhmEmpl4JlqQ3j50&confirm=t&uuid=fdc8ea42-d969-415f-8550-9b7060f16436
To: /Users/shreyadas/Downloads/Brain_Tumor_Dataset.zip
100%|██████████| 73.7M/73.7M [00:30<00:00, 2.42MB/s]


'Brain_Tumor_Dataset.zip'

In [4]:
import zipfile
import os

zip_path = 'Brain_Tumor_Dataset.zip'
extract_path = 'Brain_Tumor_Dataset'

# Check if it's a valid zip file before extracting
if zipfile.is_zipfile(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extracted files:", os.listdir(extract_path))
else:
    print("❌ The file is not a valid ZIP file.")



Extracted files: ['.DS_Store', 'Tumor', '__MACOSX']


In [20]:
batch_size = 32
img_size = (150, 150)  # Add if not already defined

dataset_path = "Brain_Tumor_Dataset"  # <-- Make sure this folder exists

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'Brain_Tumor_Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'Brain_Tumor_Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [23]:
# 🧠 Custom CNN Model (Keras)
model_keras = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(),
    BatchNormalization(),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model_keras.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_keras.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 111, 111, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 54, 54, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,908,034 (91.20 MB)

 Trainable params: 23,907,842 (91.20 MB)

 Non-trainable params: 192 (768.00 B)

In [24]:
# 🔁 Transfer Learning with ResNet50 (Keras)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model_resnet = Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,242 (90.98 MB)

 Trainable params: 262,530 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [25]:
# 🚂 Train Custom CNN (Keras)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('cnn_model.h5', save_best_only=True)
]

history = model_keras.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks
)


ValueError: The PyDataset has length 0

In [26]:
# 🧠 Custom CNN Model (PyTorch) - To be implemented in a PyTorch training loop
# Define transformations, datasets, loaders, model, criterion, optimizer, and training loop.


In [27]:
# 📊 Model Evaluation (Keras)
val_generator.reset()
preds = model_keras.predict(val_generator)
y_pred = np.argmax(preds, axis=1)
y_true = val_generator.classes

print(classification_report(y_true, y_pred, target_names=val_generator.class_indices.keys()))
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d')
plt.title("Confusion Matrix")
plt.show()


ValueError: The PyDataset has length 0

## 🚀 Streamlit Deployment Notes

- Use `cnn_model.h5` or your best model for predictions.
- Load model using `load_model()` and predict with image preprocessing.
- Include file uploader, prediction label, and confidence score in UI.

---
